In [1]:
from transformers import PhiForCausalLM, AutoTokenizer

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np

import utils
from sklearn.model_selection import train_test_split

/home/coder/.local/lib/rolos-ml-p39/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = PhiForCausalLM.from_pretrained(model_name).to(device)

In [3]:
# Get the memory reserved by PyTorch
reserved_memory = torch.cuda.memory_reserved()
reserved_memory_mb = reserved_memory / 1024**3  # Convert bytes to megabytes

print("Memory reserved by PyTorch:", reserved_memory_mb, "GB")

Memory reserved by PyTorch: 5.296875 GB


In [5]:
kotlin_code = utils.extract_kotlin_code()
train_kotlin_prompts, test_kotlin_prompts, train_kotlin_answers, test_kotlin_answers = train_test_split(*kotlin_code, test_size=0.1)

train_dataset = utils.CodeCompletionDataset(train_kotlin_prompts, train_kotlin_answers, train=True)

test_kotlin_dataset = utils.CodeCompletionDataset(test_kotlin_prompts, test_kotlin_answers, train=False)

test_codexglue_dataset = utils.CodeCompletionDataset(*utils.read_codexglue_test_data(n=1000), train=False)

Looking for kotlin files...
Parsing functions in kotlin files...


100%|██████████| 300/300 [00:03<00:00, 78.15it/s] 

parse errors count: 62, declaration errors count: 0
total number of samples: 604


In [6]:
# Get the memory reserved by PyTorch
reserved_memory = torch.cuda.memory_reserved()
reserved_memory_mb = reserved_memory / 1024**3  # Convert bytes to megabytes

print("Memory reserved by PyTorch:", reserved_memory_mb, "GB")

Memory reserved by PyTorch: 5.296875 GB


In [7]:
for key, value in utils.evaluate(model, tokenizer, test_codexglue_dataset, max_new_tokens=20).items():
    print(f"{key}: {value}")

prompt, answer = test_codexglue_dataset[0]
print(f"\nexample prompt: {prompt}\n")
print(f"example completion: {utils.sample(model, tokenizer, prompt, min_new_tokens=2, max_new_tokens=20)}\n")
print(f"example true answer: {answer}\n")

  0%|          | 3/1000 [00:04<24:46,  1.49s/it]


KeyboardInterrupt: 

In [8]:
for key, value in utils.evaluate(model, tokenizer, test_kotlin_dataset, max_new_tokens=20).items():
    print(f"{key}: {value}")

prompt, answer = test_kotlin_dataset[0]
print(f"\nexample prompt: {prompt}\n")
print(f"example completion: {utils.sample(model, tokenizer, prompt, min_new_tokens=2, max_new_tokens=20)}\n")
print(f"example true answer: {answer}\n")

 38%|███▊      | 23/61 [00:13<00:22,  1.71it/s]


KeyboardInterrupt: 

In [9]:
model = utils.train_model(model, tokenizer, train_dataset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling paral

Epoch [1/10], Loss: 0.7885:   9%|▉         | 25/272 [00:04<00:42,  5.77it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 278.00 MiB (GPU 0; 39.59 GiB total capacity; 10.26 GiB already allocated; 44.88 MiB free; 10.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
# import torch

# Get the memory reserved by PyTorch
reserved_memory = torch.cuda.memory_reserved()
reserved_memory_mb = reserved_memory / 1024**3  # Convert bytes to megabytes

print("Memory reserved by PyTorch:", reserved_memory_mb, "GB")

Memory reserved by PyTorch: 0.0 GB


In [1]:
!nvidia-smi

Sat May  4 17:30:14 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:00:08.0 Off |                    0 |
| N/A   34C    P0    41W / 250W |  36862MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
torch.cuda.empty_cache()

In [ ]:
for key, value in utils.evaluate(model, tokenizer, test_codexglue_dataset, max_new_tokens=20).items():
    print(f"{key}: {value}")

prompt, answer = test_codexglue_dataset[0]
print(f"\nexample prompt: {prompt}\n")
print(f"example completion: {utils.sample(model, tokenizer, prompt, min_new_tokens=2, max_new_tokens=20)}\n")
print(f"example true answer: {answer}\n")

In [ ]:
for key, value in utils.evaluate(model, tokenizer, test_kotlin_dataset, max_new_tokens=20).items():
    print(f"{key}: {value}")

prompt, answer = test_kotlin_dataset[0]
print(f"\nexample prompt: {prompt}\n")
print(f"example completion: {utils.sample(model, tokenizer, prompt, min_new_tokens=2, max_new_tokens=20)}\n")
print(f"example true answer: {answer}\n")

In [4]:
# model = PhiForCausalLM.from_pretrained("./fine_tuned_model").to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.34s/it]


In [31]:
# test_prompts, test_ground_truths = utils.read_codexglue_test_data()
# from nltk.translate.bleu_score import corpus_bleu
# from rouge import Rouge
# from tqdm import tqdm


# # Define function to calculate BLEU score
# def calculate_bleu_score(predictions, references):
#     return corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in predictions])

# # Define function to calculate ROUGE score
# def calculate_rouge_score(predictions, references):
#     rouge = Rouge()
#     scores = rouge.get_scores(predictions, references, avg=True)
#     return scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']

# def generate_code_completions(prompts, max_length=100):
#     generated_completions = []
#     model.eval()
#     with torch.no_grad():
#         for prompt in tqdm(prompts):
#             prompt_tensor = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
#             output = model.generate(
#                 prompt_tensor,
#                 max_new_tokens=max_length,
#                 num_return_sequences=1,
#                 pad_token_id=model.config.pad_token_id,
#                 eos_token_id=model.config.eos_token_id,
#                 bos_token_id=model.config.bos_token_id,
#             )
#             completion = tokenizer.decode(output[0], skip_special_tokens=True)
#             generated_completions.append(completion)
#     return generated_completions
# generated_code_completions = generate_code_completions(test_ground_truths[:20])
# # Calculate evaluation metrics
# accuracy = sum(1 for pred, gt in zip(generated_code_completions, test_ground_truths) if pred == gt) / len(test_ground_truths[:20])
# bleu_score = calculate_bleu_score(generated_code_completions, test_ground_truths[:20])
# rouge_score_1, rouge_score_2, rouge_score_l = calculate_rouge_score(generated_code_completions, test_ground_truths[:20])
# print("Accuracy:", accuracy)
# print("BLEU Score:", bleu_score)
# print("ROUGE Score (Rouge-1):", rouge_score_1)
# print("ROUGE Score (Rouge-2):", rouge_score_2)
# print("ROUGE Score (Rouge-L):", rouge_score_l)

model.save_pretrained("fine_tuned_model")

In [36]:
print(utils.evaluate(model, tokenizer, utils.CodeCompletionDataset(test_codexglue_dataset.prompts[:100], test_codexglue_dataset.answers[:100], train=False)))
print(utils.evaluate(model, tokenizer, utils.CodeCompletionDataset(test_kotlin_dataset.prompts[:100], test_kotlin_dataset.answers[:100], train=False)))
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from sklearn.metrics import accuracy_score

completions = []
completion_tokens = []
input_tokens = []
answers = []


def evaluate(model: torch.nn.Module, tokenizer, eval_dataset, *, max_new_tokens: int = 100):
    model.eval()
    with torch.no_grad():
        for prompt, answer in tqdm(eval_dataset):
            prompt_tensor = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
            input_tokens.append(prompt_tensor)
            output = model.generate(
                prompt_tensor,
                min_new_tokens=2,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
            )
            completion_tokens.append(output[0])
            completions.append(tokenizer.decode(output[0][len(prompt_tensor[0]):], skip_special_tokens=True))

            answers.append(answer)
    # for compl in completions:
    #     print(compl)
    #     print()

    return {
        "accuracy score": accuracy_score(answers, completions),
        "bleu score": corpus_bleu([[ans.split()] for ans in answers], [comp.split() for comp in completions]),
    }


evaluate(model, tokenizer, utils.CodeCompletionDataset(test_kotlin_dataset.prompts[:10], test_kotlin_dataset.answers[:10], train=False))
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(completions, answers, avg=True)
scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']

100%|██████████| 100/100 [01:45<00:00,  1.06s/it]


if (user.isVerbose):
        print("$DEBUG: $message")


if isSuccess(user):
        message = "[$HAS_SUCCESS] $user"
    elif isWarning(user):
        message = "[$HAS_WARNING] $user"
    elif isError(user):
        message = "[$HAS_ERROR] $user"
    else:
        message = "[$HAS_LOG] $user"
    user.l += message
    if LOG!= "console

try:
        user = self.user_lookup(request)
        if user is not None:
            response.processAll(user.receiveMessages())
        else:
            response.process("fail: user not found")
    except Exception:
        response.process("fail: exception processing")


return {
        table("user", "$name", "$auth")
        table("user", "$name", "$auth", eager = eager)
        table("user", "$name", "$auth", "caches")
    }[name]


@wraps(this)
    @require(methods) { "require(methods.isNotEmpty() == true) { " }
    @require(it == typeOf<String>()) { "Type of method is not string: $this" }
    require(it == typeOf<String?>()) { "Type of method

100%|██████████| 100/100 [00:17<00:00,  5.83it/s]




 {
    val x = test()
    if (x!= stepId) {
        return "Fail: $x!= $stepId"
    }
    return "OK"
}













("OK").value()

c + d



ilt(destination, startIndex, endIndex)





ilt(Color::class, arg)



if (targetName == null) project.targets.getOrPut(name) else JsDefaultDistribution(project, targetName)










setter(this, property, string)




ilt(first, second)

("OK").value()

("OK").value()

atan(x)

("OK").value()




args.toList()

ilt.toString()



ilt(true)







ensureEquals(actual, expected)



("OK").value()




ilt(value.toULong()).toULong()



ilt(0)

stableOpenClass.lastDefaultValueInFunction(a = n)



isFinallyMarker(node, INLINE_MARKER_FINALLY_END)














{
    var i = key.computeHash(aShift)
    while (true) {
        val k = array[i]
        if (k == null) {
            array[i] = key
            array[i + 1] = value
            return true
        }
        if (k == key) break
        if (i == 0) {
            i = array.size
        }
        i

100%|██████████| 10/10 [00:03<00:00,  3.14it/s]


(0.3028571406726984, 0.12307692215088759, 0.2980952359107937)

In [37]:
model = utils.train_model(model, tokenizer, train_dataset, epochs=8, learning_rate=1e-7, batch_size=2)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch [3/8], Loss: 0.4741:  30%|██▉       | 5983/20026 [17:08<40:18,  5.81it/s]  

In [32]:
utils.evaluate(model, tokenizer, utils.CodeCompletionDataset(test_codexglue_dataset.prompts[:100], test_codexglue_dataset.answers[:100], train=False))

100%|██████████| 100/100 [01:47<00:00,  1.07s/it]

if (user.isVerbose):
        print("$DEBUG: $message")


if isSuccess(user):
        message = "[$HAS_SUCCESS] $user"
    elif isWarning(user):
        message = "[$HAS_WARNING] $user"
    elif isError(user):
        message = "[$HAS_ERROR] $user"
    else:
        message = "[$HAS_LOG] $user"
    user.l += message
    if LOG!= "console

try:
        user = self.user_lookup(request)
        if user is not None:
            response.processAll(user.receiveMessages())
        else:
            response.process("fail: user not found")
    except Exception:
        response.process("fail: exception processing")


return {
        table("user", "$name", "$auth")
        table("user", "$name", "$auth", eager = eager)
        table("user", "$name", "$auth", "caches")
    }[name]


@wraps(this)
    @require(methods) { "require(methods.isNotEmpty() == true) { " }
    @require(it == typeOf<String>()) { "Type of method is not string: $this" }
    require(it == typeOf<String?>()) { "Type of method

{'accuracy score': 0.02, 'bleu score': 0.004466778020517986}

In [33]:
utils.evaluate(model, tokenizer, utils.CodeCompletionDataset(test_kotlin_dataset.prompts[:100], test_kotlin_dataset.answers[:100], train=False))

100%|██████████| 100/100 [00:16<00:00,  6.07it/s]



 {
    val x = test()
    if (x!= stepId) {
        return "Fail: $x!= $stepId"
    }
    return "OK"
}













("OK").value()

c + d



ilt(destination, startIndex, endIndex)





ilt(Color::class, arg)



if (targetName == null) project.targets.getOrPut(name) else JsDefaultDistribution(project, targetName)










setter(this, property, string)




ilt(first, second)

("OK").value()

("OK").value()

atan(x)

("OK").value()




args.toList()

ilt.toString()



ilt(true)







ensureEquals(actual, expected)



("OK").value()




ilt(value.toULong()).toULong()



ilt(0)

stableOpenClass.lastDefaultValueInFunction(a = n)



isFinallyMarker(node, INLINE_MARKER_FINALLY_END)














{
    var i = key.computeHash(aShift)
    while (true) {
        val k = array[i]
        if (k == null) {
            array[i] = key
            array[i + 1] = value
            return true
        }
        if (k == key) break
        if (i == 0) {
            i = array.size
        }
        i

{'accuracy score': 0.06, 'bleu score': 8.629398069945111e-06}

In [17]:
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from sklearn.metrics import accuracy_score

completions = []
completion_tokens = []
input_tokens = []
answers = []


def evaluate(model: torch.nn.Module, tokenizer, eval_dataset, *, max_new_tokens: int = 100):
    model.eval()
    with torch.no_grad():
        for prompt, answer in tqdm(eval_dataset):
            prompt_tensor = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
            input_tokens.append(prompt_tensor)
            output = model.generate(
                prompt_tensor,
                min_new_tokens=2,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
            )
            completion_tokens.append(output[0])
            completions.append(tokenizer.decode(output[0][len(prompt_tensor[0]):], skip_special_tokens=True))

            answers.append(answer)
    # for compl in completions:
    #     print(compl)
    #     print()

    return {
        "accuracy score": accuracy_score(answers, completions),
        "bleu score": corpus_bleu([[ans.split()] for ans in answers], [comp.split() for comp in completions]),
    }


evaluate(model, tokenizer, utils.CodeCompletionDataset(test_kotlin_dataset.prompts[:100], test_kotlin_dataset.answers[:100], train=False))

100%|██████████| 100/100 [00:34<00:00,  2.87it/s]


{'accuracy score': 0.04, 'bleu score': 0.05493027320889365}

In [18]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(completions, answers, avg=True)
scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']

(0.28534116965615375, 0.20281163341389366, 0.2819595708627902)